In [1]:
import branca
import geojson
import geopandas as gpd
import folium
import json
import h3
import io
import pandas as pd
import selenium
from s2sphere import CellId, LatLng, Cell
import xarray as xr

from branca.colormap import linear
from folium.plugins import MousePosition
from PIL import Image
from shapely import Polygon

In [2]:
temp_path = 'C:/Users/vladk/NC_DATA/'

In [3]:
ds = xr.open_dataset(temp_path + 'gebco_2024_n45.23_s40.85_w36.59_e42.0.nc')
ds = ds.to_dataframe().reset_index()
ds = ds.loc[ds['elevation'] <= 0]
ds

,lat,lon,elevation
17300,40.906250,38.368750,-9
18595,40.910417,38.356250,-5
18596,40.910417,38.360417,-8
18597,40.910417,38.364583,-18
18598,40.910417,38.368750,-53
...,...,...,...
1363228,45.227083,37.960417,0
1363229,45.227083,37.964583,0
1363230,45.227083,37.968750,0
1363231,45.227083,37.972917,0


In [4]:
# set sies for h3 and s2 cells (h3: edge length 8.6 km, cell area - 252.9 km^2. s2: edge length 19.5 km, cell area 324.3 km^2)
RES_H3 = 5
RES_S2 = 9

In [5]:
# function to take h3-index from coordinates 

def convert_to_h3(lat, lon, res):
    h3_index = h3.latlng_to_cell(lat, lon, res)
    return h3_index

def h3_to_table(row):
    lat = row['lat']
    lon = row['lon']
    h3_index = convert_to_h3(lat, lon, RES_H3)
    return h3_index

In [6]:
# function to get s2 index from coordinates
def s2_to_table(row):
    lat = row['lat']
    lon = row['lon']
    cell_id = CellId.from_lat_lng(LatLng.from_degrees(lat, lon)).parent(RES_S2)
    return cell_id

In [7]:
# function to get the vertices of an S2 cell
def get_s2_cell_vertices(cell_id):
    cell = Cell(cell_id)
    vertices = []
    # S2 cells have 4 vertices
    for i in range(4):  
        vertex = cell.get_vertex(i)
        latlng = LatLng.from_point(vertex)
        vertices.append((latlng.lat().degrees, latlng.lng().degrees))
    # close the polygon by repeating the first vertex
    vertices.append(vertices[0])
    return vertices

In [8]:
def s2_index(data, level):
    s2_ind = []
    for lat, lon in zip(data['lat'], data['lon']):
        cell_id = CellId.from_lat_lng(LatLng.from_degrees(lat, lon)).parent(level)
        s2_ind.append(cell_id)
    return s2_ind

### apply function to get s2 and h3 indices

In [9]:
ds['h3'] = ds.apply(h3_to_table, axis=1)

In [10]:
ds['s2'] = ds.apply(s2_to_table, axis=1)
ds.head()

,lat,lon,elevation,h3,s2
17300,40.906250,38.368750,-9,852d198ffffffff,CellId: 40636c0000000000
18595,40.910417,38.356250,-5,852d198ffffffff,CellId: 40636c0000000000
18596,40.910417,38.360417,-8,852d198ffffffff,CellId: 40636c0000000000
18597,40.910417,38.364583,-18,852d198ffffffff,CellId: 40636c0000000000
18598,40.910417,38.368750,-53,852d198ffffffff,CellId: 40636c0000000000


# Groupby H3 and S2

In [11]:
gruopby_s2 = ds.groupby('s2')['elevation'].agg('mean').reset_index()
print(gruopby_s2.shape)
gruopby_s2.head()

(544, 2)


,s2,elevation
0,CellId: 405c2c0000000000,-6.746032
1,CellId: 405d740000000000,-203.142857
2,CellId: 405d7c0000000000,-422.950748
3,CellId: 405d840000000000,-671.451688
4,CellId: 405d8c0000000000,-846.132251


In [12]:
# get centroids of s2 cells (not for this case)
gruopby_s2['s2_center'] = gruopby_s2['s2'].apply(lambda x: x.to_lat_lng()) 
gruopby_s2['lat'] = gruopby_s2['s2_center'].apply(lambda x: x.lat().degrees)
gruopby_s2['lon'] = gruopby_s2['s2_center'].apply(lambda x: x.lng().degrees)
gruopby_s2 = gruopby_s2.drop('s2_center', axis=1)
gruopby_s2.head()

,s2,elevation,lat,lon
0,CellId: 405c2c0000000000,-6.746032,42.452719,41.659935
1,CellId: 405d740000000000,-203.142857,41.851886,41.878679
2,CellId: 405d7c0000000000,-422.950748,41.759397,41.712283
3,CellId: 405d840000000000,-671.451688,41.838510,41.533093
4,CellId: 405d8c0000000000,-846.132251,41.917415,41.353298


In [13]:
gruopby_h3 = ds.groupby('h3')['elevation'].agg('mean').reset_index()
print(gruopby_h3.shape)
gruopby_h3.head()

(587, 2)


,h3,elevation
0,852c2003fffffff,-850.963636
1,852c2007fffffff,-784.128542
2,852c2013fffffff,-1499.480708
3,852c2017fffffff,-1562.995570
4,852c201bfffffff,-138.766756


In [14]:
# get centroids of h3 cells (not for this case)
gruopby_h3['h3_center'] = gruopby_h3['h3'].apply(lambda x: h3.cell_to_latlng(x)) #    old variants:   h3_to_geo      cellToLatLng 
gruopby_h3['h3_center'] = gruopby_h3['h3_center'].apply(lambda x: (x[1], x[0]))
gruopby_h3['lon'] = gruopby_h3['h3_center'].apply(lambda x: x[0])
gruopby_h3['lat']  = gruopby_h3['h3_center'].apply(lambda x: x[1])
gruopby_h3 = gruopby_h3.drop('h3_center', axis=1)
gruopby_h3.head()

,h3,elevation,lon,lat
0,852c2003fffffff,-850.963636,41.050988,41.256344
1,852c2007fffffff,-784.128542,41.184178,41.367969
2,852c2013fffffff,-1499.480708,40.850979,41.294263
3,852c2017fffffff,-1562.995570,40.984170,41.406152
4,852c201bfffffff,-138.766756,40.918009,41.144362


### Prepare geojson
h3 and s2 have other invert order of lat-lon and i need to change order to save it in geojson 

In [15]:
gruopby_s2['g_reflect'] = gruopby_s2['s2'].apply(lambda x: get_s2_cell_vertices(x))
gruopby_s2.head()

,s2,elevation,lat,lon,g_reflect
0,CellId: 405c2c0000000000,-6.746032,42.452719,41.659935,"[(42.365363255296955, 41.66658104153733), (42...."
1,CellId: 405d740000000000,-203.142857,41.851886,41.878679,"[(41.76588469518899, 41.884785238675505), (41...."
2,CellId: 405d7c0000000000,-422.950748,41.759397,41.712283,"[(41.67361862988219, 41.71869698464216), (41.7..."
3,CellId: 405d840000000000,-671.451688,41.838510,41.533093,"[(41.752562383928066, 41.539871424697346), (41..."
4,CellId: 405d8c0000000000,-846.132251,41.917415,41.353298,"[(41.83129961290025, 41.36044296593703), (41.9..."


In [16]:
result_g2 = []
for i in gruopby_s2['g_reflect']:
    iter = []
    for a in i:
        int = tuple([a[1], a[0]])
        iter.append(int)
    iter = tuple(iter)
    c = Polygon(iter)
    polygon = Polygon(iter)
    result_g2.append(c)
gruopby_s2['geometry'] = result_g2
gruopby_s2 = gruopby_s2.drop('g_reflect', axis=1)
gruopby_s2.head()

,s2,elevation,lat,lon,geometry
0,CellId: 405c2c0000000000,-6.746032,42.452719,41.659935,POLYGON ((41.66658104153733 42.365363255296955...
1,CellId: 405d740000000000,-203.142857,41.851886,41.878679,POLYGON ((41.884785238675505 41.76588469518899...
2,CellId: 405d7c0000000000,-422.950748,41.759397,41.712283,"POLYGON ((41.71869698464216 41.67361862988219,..."
3,CellId: 405d840000000000,-671.451688,41.838510,41.533093,POLYGON ((41.539871424697346 41.75256238392806...
4,CellId: 405d8c0000000000,-846.132251,41.917415,41.353298,"POLYGON ((41.36044296593703 41.83129961290025,..."


In [17]:
gruopby_h3['g_reflect'] = gruopby_h3['h3'].apply(lambda x: h3.cell_to_boundary(x))   # old variant  cellToBoundary 
gruopby_h3.head()

,h3,elevation,lon,lat,g_reflect
0,852c2003fffffff,-850.963636,41.050988,41.256344,"((41.31896349151986, 40.96207512917783), (41.2..."
1,852c2007fffffff,-784.128542,41.184178,41.367969,"((41.43064512690874, 41.09533682575575), (41.3..."
2,852c2013fffffff,-1499.480708,40.850979,41.294263,"((41.35676988928988, 40.76174730576111), (41.2..."
3,852c2017fffffff,-1562.995570,40.984170,41.406152,"((41.46871645093721, 40.89500778895924), (41.3..."
4,852c201bfffffff,-138.766756,40.918009,41.144362,"((41.206923486596665, 40.82902657588255), (41...."


In [18]:
result_g2 = []
for i in gruopby_h3['g_reflect']:
    iter = []
    for a in i:
        int = tuple([a[1], a[0]])
        iter.append(int)
    iter = tuple(iter)
    c = Polygon(iter)
    polygon = Polygon(iter)
    result_g2.append(c)
gruopby_h3['geometry'] = result_g2
gruopby_h3 = gruopby_h3.drop('g_reflect', axis=1)
gruopby_h3.head()

,h3,elevation,lon,lat,geometry
0,852c2003fffffff,-850.963636,41.050988,41.256344,"POLYGON ((40.96207512917783 41.31896349151986,..."
1,852c2007fffffff,-784.128542,41.184178,41.367969,"POLYGON ((41.09533682575575 41.43064512690874,..."
2,852c2013fffffff,-1499.480708,40.850979,41.294263,"POLYGON ((40.76174730576111 41.35676988928988,..."
3,852c2017fffffff,-1562.995570,40.984170,41.406152,"POLYGON ((40.89500778895924 41.46871645093721,..."
4,852c201bfffffff,-138.766756,40.918009,41.144362,POLYGON ((40.82902657588255 41.206923486596665...


In [19]:
# save dataframes into .geojson for use it in GIS or in other purposes
gruopby_s2 = gpd.GeoDataFrame(gruopby_s2, geometry=gruopby_s2['geometry'], crs="EPSG:4326")
gruopby_s2.to_file('s2.geojson', driver="GeoJSON")
gruopby_h3 = gpd.GeoDataFrame(gruopby_h3, geometry=gruopby_h3['geometry'], crs="EPSG:4326")
gruopby_h3.to_file('h3.geojson', driver="GeoJSON")

# MAP

Here visulaisation of dataframes (by separate polygons and using whole .geojson)

In [20]:
map_s2 = folium.Map(location=[42.5, 39.8], zoom_start=7)

In [21]:
colormap = branca.colormap.LinearColormap(colors=['cyan', 'green', 'yellow', 'orange','red'],
                            vmin=round(gruopby_s2['elevation'].min(), 2), vmax=round(gruopby_s2['elevation'].max(), 2),
                             caption='elevation')

for i, a in zip(gruopby_s2['s2'], gruopby_s2['elevation']):
    vertices = get_s2_cell_vertices(i)
    
    folium.Polygon(
        locations=vertices,
        color=colormap(a),
        weight=2,
        fill=True,
        fill_color=colormap(a),
        fill_opacity=0.5,
        popup=a,
        legend=True
    ).add_to(map_s2)

map_s2.add_child(colormap)

In [22]:
colormap = branca.colormap.LinearColormap(colors=['cyan', 'green', 'yellow', 'orange','red'],
                            vmin=round(gruopby_h3['elevation'].min(), 2), vmax=round(gruopby_h3['elevation'].max(), 2),
                             caption='depth')

start = 0
for i in range(len(gruopby_h3)):
    hex_coord = h3.cell_to_boundary(gruopby_h3['h3'].iloc[i])
    meaninig_in_point = gruopby_h3['elevation'].iloc[i]
    map_center = [hex_coord[0][0], hex_coord[0][1]]
    if start == 0:
        map_h3 = folium.Map(location=map_center, zoom_start=7, crs='EPSG3857')
        start = 1
    folium.Polygon(locations=hex_coord, color=colormap(meaninig_in_point), fill=True, fill_color=colormap(meaninig_in_point), fill_opacity=0.6, popup=round(gruopby_h3['elevation'].iloc[i], 2), legend=True).add_to(map_h3)     

map_h3.add_child(colormap)

In [23]:
# save one of the maps
img_data = map_h3._to_png(5)
img = Image.open(io.BytesIO(img_data))
img.save('08_map_h3.png')

### building maps using whole .geojson

In [24]:
with open('h3.geojson') as file:
    gj = geojson.load(file)
gj

{"crs": {"properties": {"name": "urn:ogc:def:crs:OGC:1.3:CRS84"}, "type": "name"}, "features": [{"geometry": {"coordinates": [[[40.962075, 41.318963], [40.940022, 41.2317], [41.028875, 41.169082], [41.139711, 41.193671], [41.161923, 41.280857], [41.073142, 41.343532], [40.962075, 41.318963]]], "type": "Polygon"}, "properties": {"elevation": -850.9636363636364, "h3": "852c2003fffffff", "lat": 41.25634441802905, "lon": 41.05098763342498}, "type": "Feature"}, {"geometry": {"coordinates": [[[41.095337, 41.430645], [41.073142, 41.343532], [41.161923, 41.280857], [41.272828, 41.305239], [41.295183, 41.392274], [41.206473, 41.455006], [41.095337, 41.430645]]], "type": "Polygon"}, "properties": {"elevation": -784.1285418106427, "h3": "852c2007fffffff", "lat": 41.36796925284935, "lon": 41.1841777119463}, "type": "Feature"}, {"geometry": {"coordinates": [[[40.761747, 41.35677], [40.739855, 41.26943], [40.829027, 41.206923], [40.940022, 41.2317], [40.962075, 41.318963], [40.872973, 41.381527], [4

In [25]:
gdf = gpd.read_file('h3.geojson')
gdf.head(2)

,h3,elevation,lon,lat,geometry
0,852c2003fffffff,-850.963636,41.050988,41.256344,"POLYGON ((40.96208 41.31896, 40.94002 41.2317,..."
1,852c2007fffffff,-784.128542,41.184178,41.367969,"POLYGON ((41.09534 41.43065, 41.07314 41.34353..."


In [26]:
colormap = linear.YlGn_09.scale(
    gdf['elevation'].min(), gdf['elevation'].max()
)

map_h3 = folium.Map(location=[42.5, 39.8], zoom_start=7)

folium.GeoJson(gj, 
               style_function=lambda feature: {
                    "fillColor": colormap(feature["properties"]["elevation"]),
                    "color": colormap(feature["properties"]["elevation"]),
                    "weight": 2,
                    "dashArray": "5, 5",
                    "fillOpacity": 0.7,
                    "Opacity" : 0.7,
    },
).add_to(map_h3)


formatter = "function(num) {return L.Util.formatNum(num, 3) + ' º ';};"

MousePosition(
    position="topright",
    separator=" | ",
    empty_string="NaN",
    lng_first=True,
    num_digits=20,
    prefix="Coordinates:",
    lat_formatter=formatter,
    lng_formatter=formatter,
).add_to(map_s2)

colormap.add_to(map_h3)
map_h3

In [27]:
with open('s2.geojson') as file:
    gj = geojson.load(file)
gdf = gpd.read_file('s2.geojson')

colormap = linear.YlGn_09.scale(
    gdf['elevation'].min(), gdf['elevation'].max()
)

map_s2 = folium.Map(location=[42.5, 39.8], zoom_start=7)

folium.GeoJson(gj, 
               style_function=lambda feature: {
                    "fillColor": colormap(feature["properties"]["elevation"]),
                    "color": colormap(feature["properties"]["elevation"]),
                    "weight": 2,
                    "dashArray": "5, 5",
                    "fillOpacity": 0.7,
                    "Opacity" : 0.7,
    },
).add_to(map_s2)


formatter = "function(num) {return L.Util.formatNum(num, 3) + ' º ';};"

MousePosition(
    position="topright",
    separator=" | ",
    empty_string="NaN",
    lng_first=True,
    num_digits=20,
    prefix="Coordinates:",
    lat_formatter=formatter,
    lng_formatter=formatter,
).add_to(map_s2)

colormap.add_to(map_s2)
map_s2